# 査読者に公開した論拠データを更新する
査読者に公開した論拠データを最新化するタスクです。<br>
リプライを対応の中で論拠データを編集した場合のみ、実行してください。

## リプライのためのデータを再配置する
査読者に公開している論拠データを最新のものに更新してください。<br>
査読用プロジェクトを選択し、「プロジェクトを表示する」ボタンを押下することでGakuNin RDMのプロジェクトページが表示されされます。<br>
そこで公開している論拠データを最新のものへと更新してください。

In [ ]:
# 査読用PJの「メインメニュー」を表示する
import os
import traceback

from IPython.core.display import Javascript
from IPython.display import display, clear_output
from requests.exceptions import RequestException
import panel as pn

from library.task_director import TaskDirector
from library.utils.config import connect as con_config, message as msg_config
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                UnusableVault, RepoPermissionError)
from library.utils.html.button import create_button
from library.utils.input import get_grdm_connection_parameters
from library.utils.storage_provider import grdm
from library.utils.widgets import MessageBox


notebook_name = 'update_published_argument_data.ipynb'

class AccessComponent(TaskDirector):
    """コンポーネントのメインメニューを表示するクラスです。

    Attributes:
        instance:
            _form_box (pn.WidgetBox) : フォームを格納する。
            _msg_output (MessageBox) : ユーザーに提示するメッセージを格納する。
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            grdm_url(str):GRDMのURL
            grdm(Grdm):grdmファイルのGrdmクラス

    """
    def __init__(self, working_path: str) -> None:
        """AccessComponentのコンストラクタのメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        """
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。

        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def disply_component_section(self):
        """作業中のプロジェクトの全コンポーネントへとアクセスするボタンを表示するメソッドです。"""

        self.doing_task()
        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            clear_output()
            if not self._msg_output.has_message():
                self._form_box.clear()
                # コンポーネントのプロジェクトを表示
                self.generate_component_section()

        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist:
            message = msg_config.get('form', 'project_id_not_exist').format(self.project_id)
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        self.done_task()
        # フォーム表示
        pn.extension('tabulator')
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def generate_component_section(self):
        """全コンポーネントのプロジェクトのセクションを生成するメソッドです。"""
        components = self.grdm.get_component_list(self.grdm_url, self.token, self.project_id)

        if not components:
            self._msg_output.update_warning(msg_config.get('publish_argument_data_to_reviewers', 'no_component'))
            return

        for component_id, component_title in components.items():
            # ボタンの作成
            button = self.generate_button(component_id)
            component_form_box = pn.WidgetBox()
            component_form_box.width = 900
            # フォーム定義
            title = msg_config.get('publish_argument_data_to_reviewers', 'component_title')
            component_form_box.append(f'# {title}{component_title}')
            component_form_box.append(button)

            self._form_box.append(component_form_box)

    def generate_button(self, project_id: str) -> pn.pane.HTML:
        """メンバー一覧のページを表示するボタンを作成するメソッドです。

        Args:
            project_id(str):プロジェクトID

        Returns:
            pn.pane.HTML:ボタンを表示させる。

        """
        button_width = 500
        obj = create_button(
            url=self.grdm.build_main_menu_url(self.grdm_url, project_id),
            msg=msg_config.get('publish_argument_data_to_reviewers', 'go_grdm'),
            target='_blank',
            button_width=f'{button_width}px'
        )
        return pn.pane.HTML(obj, width=button_width)

AccessComponent(working_path=os.path.abspath('__file__')).disply_component_section()

In [ ]:
# 論拠データフォルダを表示する
import os

from IPython.core.display import Javascript
from IPython.display import display
import panel as pn

from library.utils.access import open_data_folder


folder_name = 'argument_data'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 再現性を確認する
査読者に公開した論拠データが再現可能な状態であることを確認してください。<br>
再現性を確認する機能を現在開発中のため、再現性の確認は手動で行ってください。

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "update_published_argument_data.ipynb"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()